In [15]:
# Jupyter Notebook Scratch Work for NCBI Protein Search
import requests
import json
import xmltodict

In [12]:
url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=protein&id=NP_001116538"
resp = requests.get(url)
resp

<Response [200]>

In [13]:
resp.content

b'<?xml version="1.0" encoding="UTF-8" ?>\n<!DOCTYPE eSummaryResult PUBLIC "-//NLM//DTD esummary v1 20041029//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20041029/esummary-v1.dtd">\n<eSummaryResult>\n<DocSum>\n\t<Id>294862258</Id>\n\t<Item Name="Caption" Type="String">NP_001116538</Item>\n\t<Item Name="Title" Type="String">microtubule-associated protein tau isoform 6 [Homo sapiens]</Item>\n\t<Item Name="Extra" Type="String">gi|294862258|ref|NP_001116538.2|[294862258]</Item>\n\t<Item Name="Gi" Type="Integer">294862258</Item>\n\t<Item Name="CreateDate" Type="String">2008/04/09</Item>\n\t<Item Name="UpdateDate" Type="String">2021/08/22</Item>\n\t<Item Name="Flags" Type="Integer">512</Item>\n\t<Item Name="TaxId" Type="Integer">9606</Item>\n\t<Item Name="Length" Type="Integer">776</Item>\n\t<Item Name="Status" Type="String">live</Item>\n\t<Item Name="ReplacedBy" Type="String"></Item>\n\t<Item Name="Comment" Type="String"><![CDATA[  ]]></Item>\n\t<Item Name="AccessionVersion" Type="Strin

In [14]:
with open("test.xml", "wb") as f:
    f.write(resp.content)

In [17]:
with open("test.xml") as f:
    data_dict = xmltodict.parse(f.read())

data_dict

OrderedDict([('eSummaryResult',
              OrderedDict([('DocSum',
                            OrderedDict([('Id', '294862258'),
                                         ('Item',
                                          [OrderedDict([('@Name', 'Caption'),
                                                        ('@Type', 'String'),
                                                        ('#text',
                                                         'NP_001116538')]),
                                           OrderedDict([('@Name', 'Title'),
                                                        ('@Type', 'String'),
                                                        ('#text',
                                                         'microtubule-associated protein tau isoform 6 [Homo sapiens]')]),
                                           OrderedDict([('@Name', 'Extra'),
                                                        ('@Type', 'String'),
                          

In [20]:
json_data = json.dumps(data_dict)

with open("test.json", "w") as json_file:
    json_file.write(json_data)

In [21]:
with open("test.json", "r") as json_file:
    protein_data = json.load(json_file)
    
protein_data

{'eSummaryResult': {'DocSum': {'Id': '294862258',
   'Item': [{'@Name': 'Caption', '@Type': 'String', '#text': 'NP_001116538'},
    {'@Name': 'Title',
     '@Type': 'String',
     '#text': 'microtubule-associated protein tau isoform 6 [Homo sapiens]'},
    {'@Name': 'Extra',
     '@Type': 'String',
     '#text': 'gi|294862258|ref|NP_001116538.2|[294862258]'},
    {'@Name': 'Gi', '@Type': 'Integer', '#text': '294862258'},
    {'@Name': 'CreateDate', '@Type': 'String', '#text': '2008/04/09'},
    {'@Name': 'UpdateDate', '@Type': 'String', '#text': '2021/08/22'},
    {'@Name': 'Flags', '@Type': 'Integer', '#text': '512'},
    {'@Name': 'TaxId', '@Type': 'Integer', '#text': '9606'},
    {'@Name': 'Length', '@Type': 'Integer', '#text': '776'},
    {'@Name': 'Status', '@Type': 'String', '#text': 'live'},
    {'@Name': 'ReplacedBy', '@Type': 'String'},
    {'@Name': 'Comment', '@Type': 'String'},
    {'@Name': 'AccessionVersion',
     '@Type': 'String',
     '#text': 'NP_001116538.2'}]}}}

In [22]:
gid_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=sequences&id=294862258&rettype=fasta&retmode=text"
gid_resp = requests.get(gid_url)

In [23]:
gid_resp.content

b'>NP_001116538.2 microtubule-associated protein tau isoform 6 [Homo sapiens]\nMAEPRQEFEVMEDHAGTYGLGDRKDQGGYTMHQDQEGDTDAGLKESPLQTPTEDGSEEPGSETSDAKSTP\nTAEDVTAPLVDEGAPGKQAAAQPHTEIPEGTTAEEAGIGDTPSLEDEAAGHVTQEPESGKVVQEGFLREP\nGPPGLSHQLMSGMPGAPLLPEGPREATRQPSGTGPEDTEGGRHAPELLKHQLLGDLHQEGPPLKGAGGKE\nRPGSKEEVDEDRDVDESSPQDSPPSKASPAQDGRPPQTAAREATSIPGFPAEGAIPLPVDFLSKVSTEIP\nASEPDGPSVGRAKGQDAPLEFTFHVEITPNVQKEQAHSEEHLGRAAFPGAPGEGPEARGPSLGEDTKEAD\nLPEPSEKQPAAAPRGKPVSRVPQLKARMVSKSKDGTGSDDKKAKTSTRSSAKTLKNRPCLSPKHPTPGSS\nDPLIQPSSPAVCPEPPSSPKYVSSVTSRTGSSGAKEMKLKGADGKTKIATPRGAAPPGQKGQANATRIPA\nKTPPAPKTPPSSATKQVQRRPPPAGPRSERGEPPKSGDRSGYSSPGSPGTPGSRSRTPSLPTPPTREPKK\nVAVVRTPPKSPSSAKSRLQTAPVPMPDLKNVKSKIGSTENLKHQPGGGKVQIINKKLDLSNVQSKCGSKD\nNIKHVPGGGSVQIVYKPVDLSKVTSKCGSLGNIHHKPGGGQVEVKSEKLDFKDRVQSKIGSLDNITHVPG\nGGNKKIETHKLTFRENAKAKTDHGAEIVYKSPVVSGDTSPRHLSNVSSTGSIDMVDSPQLATLADEVSAS\nLAKQGL\n\n'

In [24]:
print(gid_resp.content.decode("utf-8"))

>NP_001116538.2 microtubule-associated protein tau isoform 6 [Homo sapiens]
MAEPRQEFEVMEDHAGTYGLGDRKDQGGYTMHQDQEGDTDAGLKESPLQTPTEDGSEEPGSETSDAKSTP
TAEDVTAPLVDEGAPGKQAAAQPHTEIPEGTTAEEAGIGDTPSLEDEAAGHVTQEPESGKVVQEGFLREP
GPPGLSHQLMSGMPGAPLLPEGPREATRQPSGTGPEDTEGGRHAPELLKHQLLGDLHQEGPPLKGAGGKE
RPGSKEEVDEDRDVDESSPQDSPPSKASPAQDGRPPQTAAREATSIPGFPAEGAIPLPVDFLSKVSTEIP
ASEPDGPSVGRAKGQDAPLEFTFHVEITPNVQKEQAHSEEHLGRAAFPGAPGEGPEARGPSLGEDTKEAD
LPEPSEKQPAAAPRGKPVSRVPQLKARMVSKSKDGTGSDDKKAKTSTRSSAKTLKNRPCLSPKHPTPGSS
DPLIQPSSPAVCPEPPSSPKYVSSVTSRTGSSGAKEMKLKGADGKTKIATPRGAAPPGQKGQANATRIPA
KTPPAPKTPPSSATKQVQRRPPPAGPRSERGEPPKSGDRSGYSSPGSPGTPGSRSRTPSLPTPPTREPKK
VAVVRTPPKSPSSAKSRLQTAPVPMPDLKNVKSKIGSTENLKHQPGGGKVQIINKKLDLSNVQSKCGSKD
NIKHVPGGGSVQIVYKPVDLSKVTSKCGSLGNIHHKPGGGQVEVKSEKLDFKDRVQSKIGSLDNITHVPG
GGNKKIETHKLTFRENAKAKTDHGAEIVYKSPVVSGDTSPRHLSNVSSTGSIDMVDSPQLATLADEVSAS
LAKQGL


